In [1]:
import sys
sys.path.append("..")
sys.path.append("../model")
from trajcl import TrajCL
from config import Config

In [2]:
conf = Config()

conf.dataset = 'usa_large_cell'
conf.post_value_updates()

In [3]:
model = TrajCL()

/home/sagemaker-user/.conda/envs/trajcl/lib/python3.12/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [4]:
model

TrajCL(
  (clmodel): MoCo(
    (criterion): CrossEntropyLoss()
    (encoder_q): DualSTBTimeWeighted(
      (pos_encoder): PositionalEncoding(
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (structural_attn): TransformerEncoder(
        (layers): ModuleList(
          (0-1): 2 x TransformerEncoderLayer(
            (self_attn): MultiheadAttention(
              (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
            )
            (linear1): Linear(in_features=256, out_features=2048, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
            (linear2): Linear(in_features=2048, out_features=256, bias=True)
            (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
            (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
            (dropout1): Dropout(p=0.1, inplace=False)
            (dropout2): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (spatial_attn): 

In [5]:
# from ah_databricks_data_loader import DatabricksDataLoader

# # Instantiate the DatabricksDataLoader.
# ddl = DatabricksDataLoader()

# # Load data.
# test_df = ddl.load_as_spark(schema="datascience_scratchpad", table="all_traj_test_with_pck")

In [6]:
# test_df.count()

In [7]:
# test_df.show(5)

In [8]:
# test_df.repartition(10).write.mode("overwrite").parquet("/mnt/sagemaker-nvme/data/usa/test")

In [9]:
# test_df.count()

In [10]:
from glob import glob
import pandas as pd
parquet_files = glob("/home/sagemaker-user/TrajCL/data/nyc/test/*.parquet")
total_count = 0
df_list = []
for file in parquet_files:
    df = pd.read_parquet(file)
    df_list.append(df)
    total_count += len(df)


print(f"Total records in repartitioned files: {total_count}")

Total records in repartitioned files: 1445257


In [11]:
df_list[0].head()

,userid,traj_date,timestamps,wgs_seq,employername,partition_id,pck_amt,set_type,trajlen,merc_seq,weekday,time_index_list
0,34299,2024-12-26,"[2024-12-26T00:07:27.000000000, 2024-12-26T00:...","[[-74.0597008, 40.7563587], [-74.0598405, 40.7...",u s postal service,0,5070.59,train,57,"[[-8244288.181358196, 4976470.777401652], [-82...",3,"[0, 2, 4, 5, 6, 7, 8, 10, 12, 14, 15, 16, 18, ..."
1,35404,2024-10-13,"[2024-10-13T02:49:10.000000000, 2024-10-13T03:...","[[-73.9455783, 40.7803021], [-73.9452042, 40.7...",Con edison,0,140.84,train,46,"[[-8231584.122770139, 4979990.082396612], [-82...",6,"[16, 22, 35, 39, 41, 43, 45, 48, 49, 50, 54, 5..."
2,54320,2025-05-19,"[2025-05-19T01:12:10.000000000, 2025-05-19T01:...","[[-73.9089767, 40.8332557], [-73.905857, 40.83...",Sterling Nat Bnk,0,0.00,train,33,"[[-8227509.651295921, 4987777.940954166], [-82...",0,"[7, 8, 19, 20, 24, 27, 31, 49, 52, 58, 60, 67,..."
3,54320,2025-06-10,"[2025-06-10T00:00:59.000000000, 2025-06-10T00:...","[[-73.9059267, 40.8341983], [-73.9059307, 40.8...",Sterling Nat Bnk,0,0.00,train,27,"[[-8227170.126849, 4987916.624968072], [-82271...",1,"[0, 4, 20, 27, 42, 44, 45, 47, 53, 63, 65, 70,..."
4,74093,2025-03-08,"[2025-03-08T02:09:46.000000000, 2025-03-08T02:...","[[-74.1093717454287, 40.813244574627454], [-74...",T-Mobile USA,0,1603.39,train,27,"[[-8249817.525710539, 4984834.184550401], [-82...",5,"[12, 14, 15, 16, 17, 18, 53, 54, 55, 56, 57, 1..."


In [12]:
# import pandas as pd
# test_df = pd.read_parquet("/home/sagemaker-user/TrajCL/data/parquet_files/test/nyc_df_v3_with_time/traj_test_df_v3_with_ts.parquet")

In [13]:
# userids = test_df['userid'].unique()

In [14]:
import torch
device = torch.device("cuda:0")
checkpoint_file = "/home/sagemaker-user/TrajCL/exp/usa_neg_sampling_smallqueue/ep1_batch40000"
checkpoint = torch.load(checkpoint_file)
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)

/tmp/ipykernel_10420/2978008374.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_file)


TrajCL(
  (clmodel): MoCo(
    (criterion): CrossEntropyLoss()
    (encoder_q): DualSTBTimeWeighted(
      (pos_encoder): PositionalEncoding(
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (structural_attn): TransformerEncoder(
        (layers): ModuleList(
          (0-1): 2 x TransformerEncoderLayer(
            (self_attn): MultiheadAttention(
              (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
            )
            (linear1): Linear(in_features=256, out_features=2048, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
            (linear2): Linear(in_features=2048, out_features=256, bias=True)
            (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
            (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
            (dropout1): Dropout(p=0.1, inplace=False)
            (dropout2): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (spatial_attn): 

In [22]:
from utils.traj import *
import pickle
from utils.cellspace import *

from torch.nn.utils.rnn import pad_sequence
embs_parent = pickle.load(open(Config.dataset_embs_file_parent, 'rb')).to('cpu').detach() # tensor
embs_child = pickle.load(open(Config.dataset_embs_file_child, 'rb')).to('cpu').detach() # tensor
cellspace_parent = pickle.load(open(Config.dataset_cell_file_parent, 'rb'))
cellspace_child = pickle.load(open(Config.dataset_cell_file_child, 'rb'))
hier_cellspace = HirearchicalCellSpace(cellspace_parent, cellspace_child)

max_batch_size = 512
def infer_batch(traj, time_indices):
    traj_cell_parent, traj_cell_child, traj_p, traj_timedelta = zip(*[merc2cell2(l,t, hier_cellspace) for l,t in zip(traj, time_indices)])
    # print(traj_cell)
    traj_emb_p = [torch.tensor(generate_spatial_features(t, hier_cellspace)) for t in traj_p]
    traj_emb_p = pad_sequence(traj_emb_p, batch_first = False).to(device)
    traj_emb_cell_parent = [embs_parent[list(t)] for t in traj_cell_parent]
    traj_emb_cell_child = [embs_child[list(t)] for t in traj_cell_child]
    traj_emb_cell = [a + b for a, b in zip(traj_emb_cell_parent, traj_emb_cell_child)]
    traj_emb_cell = pad_sequence(traj_emb_cell, batch_first = False).to(device)
    traj_len = torch.tensor(list(map(len, traj_p)), dtype = torch.long, device = device)
    traj_timedelta = pad_sequence([torch.log(torch.tensor(t)) for t in traj_timedelta], batch_first=False, padding_value=0).to(Config.device)
    # print(traj_emb_cell, traj_emb_p, traj_len)
    traj_embs = model.interpret(traj_emb_cell.float(), traj_emb_p.float(), traj_len, traj_timedelta)
    return traj_embs
    
def infer(traj, time_indices):
    if len(traj)> max_batch_size:
        traj_embs = []
        for i in range(0, len(traj), max_batch_size):
            traj_batch = traj[i:i+max_batch_size]
            time_indices_batch = time_indices[i:i+max_batch_size] 
            traj_embs.append(infer_batch(traj_batch, time_indices_batch))
        return torch.cat(traj_embs, dim=0)
    else:
        return infer_batch(traj, time_indices)



/home/sagemaker-user/.conda/envs/trajcl/lib/python3.12/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [23]:
from sklearn.metrics.pairwise import cosine_similarity


In [24]:
df_list[0]['time_index_list'].values[0]

array([  0,   2,   4,   5,   6,   7,   8,  10,  12,  14,  15,  16,  18,
        19,  22,  26,  27,  28,  29,  31,  32,  34,  35,  36,  39,  40,
        41,  43,  45,  50,  51,  53,  55,  56,  57,  75,  76,  77, 109,
       113, 117, 122, 124, 125, 126, 128, 129, 130, 131, 133, 134, 136,
       137, 139, 140, 142, 143], dtype=int32)

In [25]:
unique_userids = pd.concat([df['userid'] for df in df_list]).unique()
print(f"Total unique userids: {len(unique_userids)}")

unique_employernames = pd.concat([df['employername'] for df in df_list]).unique()
print(f"Total unique employernames: {len(unique_employernames)}")

Total unique userids: 25831
Total unique employernames: 13911


In [26]:
def get_data_for_userid(userid):
    df_user = pd.concat([df[df['userid']==userid] for df in df_list]).reset_index(drop=True)
    return df_user

def get_data_for_employername(df_user, employername):
    df_emp = df_user[df_user['employername']==employername].reset_index(drop=True)
    return df_emp


def get_data_for_partition_train(df_emp, partition):
    df_part = df_emp[df_emp['partition_id']==partition].reset_index(drop=True)
    df_part = df_part[~df_part['weekday'].isin([5, 6])].reset_index(drop=True) # filter out weekends
    df_part = df_part[df_part['pck_amt']>0].reset_index(drop=True) # filter out zero paycheck amount
    df_part = df_part[df_part['set_type']=='train'].reset_index(drop=True)
    return df_part

def get_data_for_partition_test(df_emp, partition):
    df_part = df_emp[df_emp['partition_id']==partition].reset_index(drop=True)
    df_part = df_part[~df_part['weekday'].isin([5, 6])].reset_index(drop=True) # filter out weekends
    df_part = df_part[df_part['set_type']=='test'].reset_index(drop=True)
    return df_part

def get_traj_and_time_data(df_part):
    traj = df_part['merc_seq'].values
    time_indices = df_part['timestamps'].values
    return traj, time_indices

In [27]:
def apply_dbscan(embs, target_min_similarity=0.85):
    from sklearn.cluster import DBSCAN     # require >= 0.85 cosine similarity
    eps = 1.0 - target_min_similarity    # cosine distance threshold
    n_embs = embs.shape[0]
    db = DBSCAN(eps=eps, min_samples=max(int(n_embs*0.3), 4), metric="cosine", n_jobs=-1).fit(embs)
    return db

In [28]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

def dbscan_predict_all(db, X_train, X_new):
    nn = NearestNeighbors(radius=db.eps, metric=db.metric).fit(X_train)
    dists, idxs = nn.radius_neighbors(X_new, return_distance=True)
    y = db.labels_
    pred = np.full(len(X_new), -1, dtype=int)
    for i, (di, ii) in enumerate(zip(dists, idxs)):
        if len(ii) == 0: 
            continue
        lbls, di = y[ii], di
        mask = lbls != -1
        if mask.any():
            pred[i] = lbls[mask][np.argmin(di[mask])]
    return pred


In [29]:
from tqdm import tqdm
model.eval()
output_dict = {}

# 0: pck_not_recv, 1: pck_recv, 2: moving, 3: data insufficient
def get_gt_and_pred_label(userid):
    user_data = get_data_for_userid(userid)
    employers = user_data['employername'].unique()
    for employer in employers:
        emp_data = get_data_for_employername(user_data, employer)
        partitions = emp_data['partition_id'].unique()
        for partition in partitions:
            # print(userid, employer, partition)
            train_part_data = get_data_for_partition_train(emp_data, partition)
            test_part_data = get_data_for_partition_test(emp_data, partition)
            gt_label  = 1 if sum(test_part_data['pck_amt'].values)>0 else 0
            if len(train_part_data)<12 or len(test_part_data)<3:
                output_dict[(userid, employer, partition)] = {'gt_label': 3, 'pred_label': 3}
            else:
                train_traj, train_time_indices = get_traj_and_time_data(train_part_data)
                train_embs = infer(train_traj, train_time_indices).detach().cpu().numpy()
                test_traj, test_time_indices = get_traj_and_time_data(test_part_data)
                test_embs = infer(test_traj, test_time_indices).detach().cpu().numpy()
                db_scan = apply_dbscan(train_embs, target_min_similarity=0.95)
                labels = db_scan.labels_                      # shape: (n_samples,)
                cluster_ids = [c for c in np.unique(labels) if c != -1]
                if len(cluster_ids)>0:
                    pred_label=0
                    # print(test_embs.shape)
                    test_labels = dbscan_predict_all(db_scan, train_embs, test_embs)  # shape: (n_test_samples,)
                    for label in test_labels:
                        if label != -1:
                            pred_label = 1
                            break
                else:
                    pred_label = 2
                output_dict[(userid, employer, partition)] = {'gt_label': gt_label, 'pred_label': pred_label}

for userid in tqdm(unique_userids):
    get_gt_and_pred_label(userid)


        


100%|██████████| 25831/25831 [13:37<00:00, 31.60it/s] 


In [30]:
# save output_dict
pickle.dump(output_dict, open("/home/sagemaker-user/TrajCL/exp/usa_neg_sampling_smallqueue/nyc_test_hyp2_output_dic_20pc.pkl", 'wb'))

In [31]:
output_dict

{(34299, 'u s postal service', 0): {'gt_label': 1, 'pred_label': 1},
 (34299, 'u s postal service', 1): {'gt_label': 0, 'pred_label': 1},
 (35404, 'Con edison', 0): {'gt_label': 0, 'pred_label': 1},
 (54320, 'Sterling Nat Bnk', 0): {'gt_label': 0, 'pred_label': 1},
 (54320, 'Medix Staffing', 0): {'gt_label': 3, 'pred_label': 3},
 (74093, 'T-Mobile USA', 0): {'gt_label': 1, 'pred_label': 2},
 (92918, 'Amazon Fulfillment', 1): {'gt_label': 3, 'pred_label': 3},
 (92918, 'Barker Consulting', 0): {'gt_label': 3, 'pred_label': 3},
 (104834, 'NYC Department of Finance', 1): {'gt_label': 1, 'pred_label': 1},
 (104834, 'NYC Department of Finance', 0): {'gt_label': 1, 'pred_label': 1},
 (104834, 'city of new york', 0): {'gt_label': 1, 'pred_label': 1},
 (131568, 'Argo realty', 1): {'gt_label': 0, 'pred_label': 1},
 (131568, 'Argo realty', 0): {'gt_label': 3, 'pred_label': 3},
 (132352, 'CARE DESIGN NY L', 0): {'gt_label': 1, 'pred_label': 1},
 (132352, 'care design ny', 0): {'gt_label': 0, 'pred

In [47]:
import pickle
# output_dict = pickle.load(open("/home/sagemaker-user/TrajCL/exp/v2.1/nyc_test_hyp2_output_dict.pkl", 'rb'))
gt_label_list = pickle.load(open("/home/sagemaker-user/TrajCL/exp/usa_neg_sampling_smallqueue/gt_label_0_list_with_amt.pkl", "rb"))
inactive_user = pickle.load(open("/home/sagemaker-user/TrajCL/exp/v2.1/inactive_dict.pkl", "rb"))

FileNotFoundError: [Errno 2] No such file or directory: '/home/sagemaker-user/TrajCL/exp/v2.1/inactive_dict.pkl'

In [39]:
gt_label_0_dict = {}
for key, v in gt_label_list:
    gt_label_0_dict[key] = v

In [40]:
gt_labels = []
pred_labels = []
userids = []
for key in output_dict.keys():
    gt_label = output_dict[key]['gt_label']
    pred_label = output_dict[key]['pred_label']
    if key in gt_label_0_dict:
        amt = gt_label_0_dict[key]
        if amt>0:
            gt_label = 1

    # if key in inactive_user:
    #     gt_label = 2

    user_id = key[0]
    if gt_label in [0, 1] and pred_label in [0, 1]:
        gt_labels.append(gt_label)
        pred_labels.append(pred_label)
        userids.append(user_id)

len(gt_labels), len(pred_labels), len(set(userids))


(8266, 8266, 6468)

In [41]:
# gt_label_0_dict[(614786, 'R 2 Logistics LLC', 0)]

In [42]:
from sklearn.metrics import confusion_matrix

# Calculate confusion matrix
cm = confusion_matrix(gt_labels, pred_labels)
# Assuming you already have the confusion matrix 'cm'
# For binary classification, cm is in the form:
# [[TN, FP],
#  [FN, TP]]

tn, fp, fn, tp = cm.ravel()
print("True Positives:", tp)
print("False Positives:", fp)
print("True Negatives:", tn)
print("False Negatives:", fn)

True Positives: 7002
False Positives: 539
True Negatives: 164
False Negatives: 561


In [43]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Calculate precision and recall
precision = precision_score(gt_labels, pred_labels)
recall = recall_score(gt_labels, pred_labels)
f1 = f1_score(gt_labels, pred_labels)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Precision: 0.9285240684259382
Recall: 0.9258230860769536
F1 Score: 0.9271716101694916


In [44]:
from sklearn.metrics import precision_score, recall_score

# For both classes (binary), use average=None to get per-class metrics
precision_per_class = precision_score(gt_labels, pred_labels, average=None)
recall_per_class = recall_score(gt_labels, pred_labels, average=None)

print("Precision for each class:", precision_per_class)
print("Recall for each class:", recall_per_class)

Precision for each class: [0.2262069  0.92852407]
Recall for each class: [0.23328592 0.92582309]


In [45]:
sum(gt_labels)/len(gt_labels), sum(pred_labels)/len(pred_labels)

(0.9149528187757077, 0.9122913138156303)

In [46]:
accuracy = (tp + tn) / (tp + tn + fp + fn)
print("Accuracy:", accuracy)

Accuracy: 0.8669247519961287


In [28]:
for key in output_dict.keys():
    gt_label = output_dict[key]['gt_label']
    pred_label = output_dict[key]['pred_label']
    if key in gt_label_0_dict:
        amt = gt_label_0_dict[key]
        if amt>0:
            gt_label = 1
    
    if key in inactive_user:
        gt_label = 2
    if gt_label == 0 and pred_label == 1:
        print(key)

(526137, 'uber driver', 0)
(614786, 'R 2 Logistics LLC', 0)
(1017879, 'BGB Group', 0)
(4968864, 'West New York Board of Education', 1)
(5401643, 'MDB Shipping inc', 0)
(6048564, 'Ridgefield park school district', 0)
(6048564, 'Ridgefield park school district', 1)
(6269644, 'TJX Companies', 0)
(7694918, 'star and son electrical supply', 0)
(7826038, 'Black Veterans for Social Justice, Inc.', 0)
(9363424, 'Blackrock Invest', 0)
(14205175, 'Acme fresh market', 0)
(14219458, 'Military', 1)
(14542515, 'sunglasses hut', 0)
(15277615, 'Urban Renewal Corp', 0)
(15894950, 'New Fulton Fish Market', 1)
(16814917, 'Hostos Community College', 0)
(16829055, 'John starks kia ', 0)
(17230232, 'Chase', 1)
(17625951, 'Securitas Security Services ', 0)
(17652714, 'Gnana Sunderam MD ', 1)
(17984542, 'ermc aviation', 0)
(18013499, 'The Newark Museum of Art ', 1)
(18064975, 'CALL-A-HEAD', 0)
(18733351, 'Amazon', 0)
(18736759, 'Chipotle Mexican Grill', 1)
(18762027, 'Pls Check Casher', 0)
(18773734, 'preferr

In [41]:
def get_data_userid_employer_partition(userid, employer, partition):
    df_user = pd.concat([df[df['userid']==userid] for df in df_list]).reset_index(drop=True)
    df_emp = df_user[df_user['employername']==employer].reset_index(drop=True)
    df_part = df_emp[df_emp['partition_id']==partition].reset_index(drop=True)
    df_part_fil = df_part[~df_part['weekday'].isin([5, 6])].reset_index(drop=True) # filter out weekends
    train_df = df_part_fil[df_part_fil['pck_amt']>0].reset_index(drop=True) # filter out zero paycheck amount
    train_df = train_df[train_df['set_type']=='train'].reset_index(drop=True)
    test_df = df_part_fil[df_part_fil['set_type']=='test'].reset_index(drop=True)
    
    return train_df, test_df

In [71]:
train_df, test_df = get_data_userid_employer_partition(18733351, 'Amazon', 0)

In [72]:
train_df.sort_values('traj_date')

,userid,traj_date,timestamps,wgs_seq,employername,partition_id,pck_amt,set_type,trajlen,merc_seq,weekday,time_index_list
97,18733351,2024-06-27,"[2024-06-27T00:05:03.000000000, 2024-06-27T00:...","[[-73.7664409, 40.7119492], [-73.7666953, 40.7...",Amazon,0,327.48,train,39,"[[-8211642.638620109, 4969946.630016507], [-82...",3,"[0, 1, 2, 3, 5, 7, 11, 23, 46, 48, 49, 50, 52,..."
52,18733351,2024-06-28,"[2024-06-28T00:03:59.000000000, 2024-06-28T00:...","[[-73.7664315, 40.7123304], [-73.7664934, 40.7...",Amazon,0,327.48,train,74,"[[-8211641.592216896, 4970002.613136906], [-82...",4,"[0, 4, 6, 8, 10, 17, 29, 31, 37, 39, 40, 41, 4..."
82,18733351,2024-07-01,"[2024-07-01T00:00:11.000000000, 2024-07-01T00:...","[[-73.766578, 40.7121054], [-73.7652528, 40.71...",Amazon,0,327.48,train,49,"[[-8211657.900522296, 4969969.569545971], [-82...",0,"[0, 1, 2, 5, 6, 7, 8, 9, 11, 12, 13, 15, 16, 1..."
96,18733351,2024-07-02,"[2024-07-02T00:00:52.000000000, 2024-07-02T00:...","[[-73.7660391, 40.7123427], [-73.7664081, 40.7...",Amazon,0,327.48,train,27,"[[-8211597.910448709, 4970004.419523098], [-82...",1,"[0, 1, 2, 5, 7, 8, 13, 14, 15, 17, 21, 24, 47,..."
110,18733351,2024-07-03,"[2024-07-03T00:12:05.000000000, 2024-07-03T00:...","[[-73.766766, 40.7121687], [-73.7660253, 40.71...",Amazon,0,637.08,train,68,"[[-8211678.828586566, 4969978.865798268], [-82...",2,"[1, 2, 3, 4, 23, 47, 52, 54, 55, 56, 57, 59, 6..."
...,...,...,...,...,...,...,...,...,...,...,...,...
95,18733351,2024-12-05,"[2024-12-05T00:01:41.000000000, 2024-12-05T03:...","[[-73.7659585, 40.7123322], [-73.7659947, 40.7...",Amazon,0,669.40,train,19,"[[-8211588.938097751, 4970002.877486084], [-82...",3,"[0, 18, 42, 47, 48, 49, 51, 125, 127, 128, 132..."
79,18733351,2024-12-06,"[2024-12-06T00:02:06.000000000, 2024-12-06T00:...","[[-73.7659823, 40.7123454], [-73.7659827, 40.7...",Amazon,0,669.40,train,27,"[[-8211591.587501631, 4970004.816046938], [-82...",4,"[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 12, 13, 36, 51..."
91,18733351,2024-12-09,"[2024-12-09T00:05:59.000000000, 2024-12-09T00:...","[[-73.7662779, 40.7122999], [-73.7660303, 40.7...",Amazon,0,669.40,train,22,"[[-8211624.493543111, 4969998.133888037], [-82...",0,"[0, 2, 3, 4, 6, 8, 14, 15, 16, 37, 42, 48, 50,..."
45,18733351,2024-12-10,"[2024-12-10T00:08:20.000000000, 2024-12-10T00:...","[[-73.7661239, 40.7122674], [-73.7658256, 40.7...",Amazon,0,669.40,train,34,"[[-8211607.350341527, 4969993.360920191], [-82...",1,"[0, 3, 7, 11, 12, 14, 16, 22, 34, 47, 49, 50, ..."


In [73]:
test_df.sort_values('traj_date')

,userid,traj_date,timestamps,wgs_seq,employername,partition_id,pck_amt,set_type,trajlen,merc_seq,weekday,time_index_list
3,18733351,2024-12-19,"[2024-12-19T00:04:14.000000000, 2024-12-19T00:...","[[-73.7660066, 40.7123328], [-73.7660209, 40.7...",Amazon,0,0.00,test,24,"[[-8211594.292565257, 4970002.965602478], [-82...",3,"[0, 1, 3, 5, 8, 9, 11, 16, 17, 28, 48, 49, 51,..."
0,18733351,2024-12-20,"[2024-12-20T00:03:15.000000000, 2024-12-20T00:...","[[-73.7659793, 40.7123336], [-73.7659764, 40.7...",Amazon,0,0.00,test,24,"[[-8211591.253543159, 4970003.083091006], [-82...",4,"[0, 1, 5, 11, 22, 24, 29, 42, 43, 44, 46, 48, ..."
2,18733351,2024-12-23,"[2024-12-23T00:22:36.000000000, 2024-12-23T00:...","[[-73.7661379, 40.7123092], [-73.7665714, 40.7...",Amazon,0,0.00,test,31,"[[-8211608.908814399, 4969999.499691574], [-82...",0,"[2, 3, 4, 6, 8, 12, 13, 15, 17, 18, 20, 22, 26..."
1,18733351,2024-12-24,"[2024-12-24T00:01:04.000000000, 2024-12-24T00:...","[[-73.766273, 40.7121437], [-73.7659812, 40.71...",Amazon,0,0.00,test,63,"[[-8211623.948077604, 4969975.194291563], [-82...",1,"[0, 3, 5, 6, 8, 9, 10, 15, 16, 18, 19, 21, 22,..."
4,18733351,2024-12-25,"[2024-12-25T00:04:19.000000000, 2024-12-25T00:...","[[-73.7662722, 40.7123369], [-73.7663364, 40.7...",Amazon,0,0.00,test,22,"[[-8211623.859022013, 4970003.567731192], [-82...",2,"[0, 3, 23, 24, 43, 44, 46, 47, 48, 49, 50, 52,..."


In [74]:
def get_gt_and_pred_label(train_df, test_df):
    userid = train_df['userid'].values[0]
    employer = train_df['employername'].values[0]
    train_traj = train_df['merc_seq'].values
    test_traj = test_df['merc_seq'].values
    train_time_indices = train_df['time_index_list'].values
    test_time_indices = test_df['time_index_list'].values
    train_embs = infer(train_traj, train_time_indices).detach().cpu().numpy()
    test_embs = infer(test_traj, test_time_indices).detach().cpu()
    db_scan = apply_dbscan(train_embs, target_min_similarity=0.85)
    core_points = db_scan.core_sample_indices_
    print(f"Core points : ", train_df.iloc[core_points][['traj_date', 'pck_amt']])
    labels = db_scan.labels_
    for i in range(len(test_embs)):
        # print("Test Trajectory:", test_df['traj_date'].values[i])
        test_vector = test_embs[i].unsqueeze(0)
        similarity = cosine_similarity(test_vector.numpy(), train_embs)[0]
        top_3_indices = np.argsort(similarity)[-3:][::-1]
        bottom_3_indices = np.argsort(similarity)[:3]
        # print(i, top_3_indices)
        top_similarity = similarity[top_3_indices]
        bottom_similarity = similarity[bottom_3_indices]
        print(f"User: {userid}, Test Trajectory {test_df['traj_date'].values[i]}, Top 3 Train Trajectories: {train_df['traj_date'].values[top_3_indices]}, similarity: {top_similarity}, PCK Amount: {train_df['pck_amt'].values[top_3_indices]}, Cluster Labels: {labels[top_3_indices]}")
        # print(f"Bottom 3 Train Trajectories: {train_df['traj_date'].values[bottom_3_indices]}, similarity: {bottom_similarity}, PCK Amount: {train_df['pck_amt'].values[bottom_3_indices]}")
        print("------------")


In [75]:
get_gt_and_pred_label(train_df, test_df)

Core points :        traj_date pck_amt
0    2024-09-09  783.33
1    2024-09-25  660.18
2    2024-11-06  506.53
3    2024-08-14  738.73
4    2024-11-04  506.53
..          ...     ...
103  2024-07-16  646.77
105  2024-09-30  671.58
106  2024-10-11  690.68
109  2024-12-03  848.71
111  2024-10-24  675.77

[74 rows x 2 columns]
User: 18733351, Test Trajectory 2024-12-20, Top 3 Train Trajectories: [datetime.date(2024, 9, 9) datetime.date(2024, 10, 1)
 datetime.date(2024, 10, 17)], similarity: [0.975036   0.97056794 0.96814686], PCK Amount: [Decimal('783.33') Decimal('671.58') Decimal('673.93')], Cluster Labels: [0 0 0]
------------
User: 18733351, Test Trajectory 2024-12-24, Top 3 Train Trajectories: [datetime.date(2024, 7, 24) datetime.date(2024, 11, 13)
 datetime.date(2024, 9, 18)], similarity: [0.8639975  0.85795456 0.85071695], PCK Amount: [Decimal('752.28') Decimal('626.67') Decimal('625.18')], Cluster Labels: [0 0 0]
------------
User: 18733351, Test Trajectory 2024-12-23, Top 3 Train

In [42]:
train_df

,userid,traj_date,timestamps,wgs_seq,employername,partition_id,pck_amt,set_type,trajlen,merc_seq,weekday,time_index_list
0,526137,2024-11-21,"[2024-11-21T05:17:26.000000000, 2024-11-21T05:...","[[-74.15075987581861, 40.91694194881615], [-74...",uber driver,0,1587.41,train,47,"[[-8254424.831310429, 5000098.328570914], [-82...",3,"[31, 35, 36, 37, 38, 43, 44, 45, 46, 47, 49, 5..."
1,526137,2025-03-28,"[2025-03-28T04:48:14.000000000, 2025-03-28T04:...","[[-74.13188592448114, 40.88513793263814], [-74...",uber driver,0,925.88,train,37,"[[-8252323.792658284, 4995414.2690154305], [-8...",4,"[28, 29, 30, 31, 32, 34, 35, 36, 37, 38, 39, 4..."
2,526137,2024-12-04,"[2024-12-04T05:13:09.000000000, 2024-12-04T05:...","[[-74.1536444272704, 40.91728962572706], [-74....",uber driver,0,798.59,train,39,"[[-8254745.938109209, 5000149.546482364], [-82...",2,"[31, 34, 35, 36, 39, 40, 41, 43, 44, 45, 46, 4..."
3,526137,2025-03-20,"[2025-03-20T05:00:20.000000000, 2025-03-20T05:...","[[-74.16809442676049, 40.89149568295617], [-74...",uber driver,0,1061.97,train,40,"[[-8256354.50469441, 4996350.451204671], [-824...",3,"[30, 31, 32, 33, 34, 36, 37, 38, 39, 44, 51, 5..."
4,526137,2024-11-22,"[2024-11-22T05:09:31.000000000, 2024-11-22T05:...","[[-74.15172639757753, 40.91744589167505], [-74...",uber driver,0,1587.41,train,35,"[[-8254532.424020474, 5000172.566837596], [-82...",4,"[30, 32, 37, 38, 39, 40, 41, 42, 43, 44, 45, 4..."
...,...,...,...,...,...,...,...,...,...,...,...,...
74,526137,2025-02-21,"[2025-02-21T05:00:44.000000000, 2025-02-21T05:...","[[-74.15361809831225, 40.917327198728245], [-7...",uber driver,0,1886.95,train,26,"[[-8254743.007182995, 5000155.081552985], [-82...",4,"[30, 31, 37, 38, 39, 40, 41, 42, 43, 44, 45, 4..."
75,526137,2024-12-13,"[2024-12-13T05:48:11.000000000, 2024-12-13T05:...","[[-74.16701113175779, 40.90699845622197], [-74...",uber driver,0,479.33,train,28,"[[-8256233.91284633, 4998633.6201853], [-82434...",4,"[34, 35, 36, 37, 38, 39, 40, 43, 44, 45, 51, 5..."
76,526137,2024-12-17,"[2024-12-17T05:38:37.000000000, 2024-12-17T05:...","[[-74.15121081346709, 40.8789008817892], [-74....",uber driver,0,1526.72,train,42,"[[-8254475.029459838, 4994495.947278066], [-82...",1,"[33, 34, 36, 37, 38, 39, 40, 41, 43, 44, 45, 4..."
77,526137,2024-11-25,"[2024-11-25T05:23:48.000000000, 2024-11-25T05:...","[[-74.15013459371903, 40.916869916504254], [-7...",uber driver,0,1587.41,train,35,"[[-8254355.225225502, 5000087.717187921], [-82...",0,"[32, 33, 36, 37, 38, 40, 41, 42, 43, 44, 45, 4..."


In [143]:
import datetime
def get_similarity_between_two_traj(df, traj_date_1, traj_date_2):
    date_1 = datetime.datetime.strptime(traj_date_1, '%Y-%m-%d').date()
    date_2 = datetime.datetime.strptime(traj_date_2, '%Y-%m-%d').date()
    traj_1 = df[df['traj_date']==date_1]['merc_seq'].values[0]
    time_indices_1 = df[df['traj_date']==date_1]['time_index_list'].values[0]
    traj_2 = df[df['traj_date']==date_2]['merc_seq'].values[0]
    time_indices_2 = df[df['traj_date']==date_2]['time_index_list'].values[0]
    emb_1 = infer([traj_1], [time_indices_1]).detach().cpu().numpy()
    emb_2 = infer([traj_2], [time_indices_2]).detach().cpu().numpy()
    similarity = cosine_similarity(emb_1, emb_2)[0][0]
    return similarity

get_similarity_between_two_traj(pd.concat([train_df, test_df]).reset_index(drop=True), '2025-07-16', '2025-06-03')

0.6525289